In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
import logging


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
from math_rag.infrastructure.containers import InfrastructureContainer


RESET = False

# containers
infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()
infrastructure_container.wire(modules=[__name__])

application_container = infrastructure_container.application_container()
application_container.init_resources()
application_container.wire(modules=[__name__])

# seed
for object_seeder in infrastructure_container.object_seeders():
    object_seeder.seed(reset=RESET)

for document_seeder in infrastructure_container.document_seeders():
    await document_seeder.seed(reset=RESET)

for embedding_seeder in infrastructure_container.embedding_seeders():
    await embedding_seeder.seed(reset=RESET)

# index
for document_indexer in infrastructure_container.document_indexers():
    await document_indexer.index(reset=RESET)

2025-06-11 10:28:23,804 - INFO - PyTorch version 2.6.0 available.
2025-06-11 10:28:24,116 - INFO - HTTP Request: GET http://qdrant:6333 "HTTP/1.1 200 OK"
2025-06-11 10:28:24,120 - INFO - HTTP Request: GET http://qdrant:6333/collections/mathexpressiondescriptionembedding/exists "HTTP/1.1 200 OK"


In [4]:
fine_tune_job_runner_service = infrastructure_container.fine_tune_job_runner_service()

In [5]:
await fine_tune_job_runner_service.init_resources()

2025-06-11 10:28:24,633 - INFO - Host canonicalization disabled
2025-06-11 10:28:24,635 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-06-11 10:28:24,690 - INFO - [conn=0] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-06-11 10:28:24,691 - INFO - [conn=0]   Local address: 172.18.0.11, port 58576
2025-06-11 10:28:24,691 - INFO - [conn=0]   Peer address: 161.53.2.37, port 22
2025-06-11 10:28:24,728 - INFO - [conn=0] Beginning auth for user lpanic
2025-06-11 10:28:25,072 - INFO - [conn=0] Auth for user lpanic succeeded
2025-06-11 10:28:25,075 - INFO - [conn=0, chan=0] Requesting new SSH session
2025-06-11 10:28:25,085 - INFO - [conn=0, chan=0]   Command: mkdir -p lora_default_root
2025-06-11 10:28:25,249 - INFO - [conn=0, chan=0] Received exit status 0
2025-06-11 10:28:25,250 - INFO - [conn=0, chan=0] Received channel close
2025-06-11 10:28:25,251 - INFO - [conn=0, chan=0] Channel closed
2025-06-11 10:28:25,252 - INFO - Command `mkdir -p lora_defaul

In [6]:
from math_rag.core.models import FineTuneJob


fine_tune_job = FineTuneJob(provider_name='meta-llama', model_name='Llama-3.1-8B')

In [7]:
await fine_tune_job_runner_service.run(fine_tune_job, poll_interval=5 * 60)

2025-06-11 10:28:39,996 - INFO - Host canonicalization disabled
2025-06-11 10:28:39,996 - INFO - Opening SSH connection to login-gpu.hpc.srce.hr, port 22
2025-06-11 10:28:40,013 - INFO - [conn=28] Connected to SSH server at login-gpu.hpc.srce.hr, port 22
2025-06-11 10:28:40,013 - INFO - [conn=28]   Local address: 172.18.0.11, port 44728
2025-06-11 10:28:40,014 - INFO - [conn=28]   Peer address: 161.53.2.37, port 22
2025-06-11 10:28:40,047 - INFO - [conn=28] Beginning auth for user lpanic
2025-06-11 10:28:40,318 - INFO - [conn=28] Auth for user lpanic succeeded
2025-06-11 10:28:40,321 - INFO - [conn=28, chan=0] Requesting new SSH session
2025-06-11 10:28:40,356 - INFO - [conn=28, chan=0]   Subsystem: sftp
2025-06-11 10:28:40,365 - INFO - [conn=28, chan=0] Starting SFTP client
2025-06-11 10:28:40,554 - INFO - [conn=28, chan=0] Received exit status 0
2025-06-11 10:28:40,556 - INFO - [conn=28, chan=0] Received channel close
2025-06-11 10:28:40,558 - INFO - [conn=28, chan=0] SFTP client exi

CancelledError: 